In [1]:
#!/usr/bin/env python
import sys 

import os 
import pandas as pd
from wmf import wmf
import numpy as np 
import glob 
import pylab as pl
import json
import MySQLdb
import csv
import matplotlib
import matplotlib.font_manager
from datetime import timedelta
import datetime as dt
import pickle
import matplotlib.dates as mdates
import alarmas as al
import simplekml
import pykml

from cpr import cpr

import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline

### Funciones

In [3]:
def mappingConfigtable(col_df):
    ''' #Ejecuta funcion map(int,) sobre 'col_df' seteando los vacios para poderlos leer posteriormente.
        #Funcion base.
        #Argumentos:
        col_df: df[column] del dfconfig.
    ''' 
    lis=list(col_df)
    for pos,value in enumerate(lis):
        if value=='':
            lis[pos]=''
        elif len(value.split(','))>0:
            lis[pos]=map(int,value.split(','))
        else:
            lis[pos]=map(int,value.split(','))
    return lis

In [4]:
# entradas
rutaconfig='/media/nicolas/Home/Jupyter/Soraya/git/Alarmas/04_web_hidrologia/hidrologia/informacion_sirenas.md'
listconfig=al.get_rutesList(rutaconfig)

#se trae la informacion de las estaciones desde CPR en un df.
self=cpr.Nivel(90) #--> cualquier estacion para definir la clase.
#df
infoestaciones = self.infost
risklevels_all=infoestaciones[['action_level','minor_flooding','moderate_flooding','major_flooding']]

## Sirenas
### Diccionario estático con todo lo de sirenas-alarmas.

- Nombre alarmas
- Nombre corrientes
- Código niveles asociadas
- KML de cuenca asociada para cada una.
- (...KML red de drenaje...)
- Mapas ubicación general: KMLs ubicacion de alarmas, KML microcuencas, KML municipios.

**Nota:**
- Las coordenadas de las alarmas con sus nombres, están en este msg:
     
        '../series/op/nombres_sirenas_op.msg'
Este se va a actualizar sólo cuando se agreguen nuevas estaciones de nivel asociadas a alarmas, la mayoría del tiempo es estático.


- Se incluyen las estaciones de nivel que están sobre la sirena o aguas arriba.

In [5]:
def set_dfconfig2Json(rutaconfig,verbose=True):
    ''' #Crea un JSON con la informacion de cada cuenca: estaciones de redes nivel, pluvio, meteo y kml's de cuencas.
        Las estaciones asignadas y las rutas de los kml se sacan del configfile. 
        #Funcion no operacional.
        #Argumentos:
        rutaconfig: ruta del configfile.
    '''
    # rutaconfig='informacion_sirenas.md'
    listconfig=al.get_rutesList(rutaconfig)
    List=al.get_tables(listconfig,'-al')    
    dfconfig=pd.DataFrame.from_records([i[1:-2].split('|') for i in List])
    dfconfig.columns=dfconfig.iloc[0]
    dfconfig.index=dfconfig['Codigo']
    dfconfig=dfconfig.drop('Codigo')
    dfconfig=dfconfig.drop('-al',axis=1)
    dfconfig=dfconfig.drop('Codigo',axis=1)
    dfconfig=dfconfig.drop(dfconfig.columns[-1],axis=1)
    dfconfig.index.name=''
    #####ASIGNACION DE ESTACIONES.
    niveles_sirenas= al.mappingConfigtable(dfconfig['Est_Nivel'])#[[90,108],[173],[173],[259],[251,186],[265],[179],[246]]
    niveles1_sirenas=al.mappingConfigtable(dfconfig['Est_Nivel1'])
    pluvioadentro_sirenas=al.mappingConfigtable(dfconfig['Est_PluvioAdentro'])
    pluvioafuera_sirenas=al.mappingConfigtable(dfconfig['Est_PluvioAfuera'])
    meteoadentro_sirenas=al.mappingConfigtable(dfconfig['Est_MeteoAdentro'])
    meteoafuera_sirenas=al.mappingConfigtable(dfconfig['Est_MeteoAfuera'])
    N30m_sirenas=al.mappingConfigtable(dfconfig['Est_N30m'])
    Tto_sirenas=al.mappingConfigtable(dfconfig['Est_Tto'])
    #la cuenca La Lopez, está contenida en La Gallinaza - Barbosa, por eso se deja la ultima como la asociada a la alarma.
    nombres_corrientes=list(dfconfig['Nombre_Quebrada'])#np.array(['Q. Dona Maria', 'Q. La Picacha','Q. La Picacha', 'Q. La Corrala', 'Q. La Gallinaza', 'Q. La Loca', 'Rio Medellin - La Inmaculada', 'Q. La Raya'])
    nombres_sirenas=list(dfconfig['Nombre_Sirena'])
    codigos_sirenas=list(dfconfig.index)

    #Se toman las direcciones de los .kml de cada cuenca.
    urlcuencas=glob.glob(al.get_ruta(listconfig,'ruta_KMLs')+'cuencas_alarmas/*')

    # se crean dicts.
    names_sirenas={};names_corrientes={};niv_sirenas={};niv1_sirenas={};pluvioadentro={};pluvioafuera={};meteoadentro={};meteoafuera={}
    N30m={};Tto={};url_cuencas={}
    for ind,sir in enumerate(codigos_sirenas):
        #se obtiene y guarda los datos de las n_alarmas
        names_sirenas.update({sir:nombres_sirenas[ind]})
        names_corrientes.update({sir:nombres_corrientes[ind]})
        niv_sirenas.update({sir:niveles_sirenas[ind]})
        niv1_sirenas.update({sir:niveles1_sirenas[ind]})
        pluvioadentro.update({sir:pluvioadentro_sirenas[ind]})
        pluvioafuera.update({sir:pluvioafuera_sirenas[ind]})
        meteoadentro.update({sir:meteoadentro_sirenas[ind]})
        meteoafuera.update({sir:meteoafuera_sirenas[ind]})
        N30m.update({sir:N30m_sirenas[ind]})
        Tto.update({sir:Tto_sirenas[ind]})
#         url_cuencas.update({sir:urlcuencas[ind]})

    #Escribe diccionario para json
    dictt={}
    dictt.update({'Nombres Alarmas':names_sirenas})
    dictt.update({'Nombres Corriente Asociada':names_corrientes})
    dictt.update({'EstNivel':niv_sirenas})
    dictt.update({'EstNivel1':niv1_sirenas})
    dictt.update({'EstPluvioAdentro':pluvioadentro})
    dictt.update({'EstPluvioAfuera':pluvioafuera})
    dictt.update({'EstMeteoAdentro':meteoadentro})
    dictt.update({'EstMeteoAfuera':meteoafuera})
    dictt.update({'EstN30m':N30m})
    dictt.update({'EstTto':Tto})
    dictt.update({'URL Cuencas':url_cuencas})

    #Se crea dataframe para guardar porque desde el Dic no sirve.
    df=pd.DataFrame.from_dict(dictt)
    #se guarda
    if verbose:
        try:
            ruta_out=al.get_ruta(listconfig,'ruta_JSONinfosirenas')
            df.to_json(ruta_out)
            print 'Aviso: se crea JSON en '+ruta_out    
        except:
            print 'Aviso: no se crea JSON, revisar!'

In [6]:
set_dfconfig2Json(rutaconfig)

Aviso: se crea JSON en /media/nicolas/Home/Jupyter/Soraya/git/Alarmas/04_web_hidrologia/hidrologia/results/01_estaticos/infosirenas.json


In [7]:
dfconfig=pd.read_json(al.get_ruta(listconfig,'ruta_JSONinfosirenas'))
dfconfig

,EstMeteoAdentro,EstMeteoAfuera,EstN30m,EstNivel,EstNivel1,EstPluvioAdentro,EstPluvioAfuera,EstTto,Nombres Alarmas,Nombres Corriente Asociada,URL Cuencas
216,[197],[249],[108],"[90, 108]",[108],"[3, 18, 43]",,,SIRENA SANTA RITA SAN ANTONIO DE PRADO,Q. DONA MARIA,NaN
219,,[249],,"[173, 116]",[173],[29],"[1, 8]",,SIRENA BELEN - LAS VIOLETAS,Q. LA PICACHA,NaN
220,,[249],,"[173, 116]",[116],[29],"[1, 8, 9]",,SIRENA AGUAS FRIAS,Q. PICACHA,NaN
221,,[105],,[259],[259],,"[33, 253]",,SIRENA ANDALUCIA CALDAS,Q. LA CORRALA,NaN
222,,[82],,"[251, 186]",[186],,"[234, 30]",,SIRENA BARBOSA,Q. LA GALLINAZA,NaN
223,,,,"[265, 135]",[135],,"[12, 14, 48, 89]",,SIRENA BELLO CAFETAL,Q. LA LOCA,NaN
224,[105],,"[179, 106]","[179, 106, 124]",[106],"[267, 61, 261, 33, 253]",,[179],SIRENA LA ESTRELLA,RIO MEDELLIN - LA INMACULADA,NaN
226,,[105],,[246],[246],[261],[61],,SIRENA LA RAYA,Q. LA RAYA,NaN
279,,[70],,[239],[239],[248],[70],,SIRENA PIEDRAS BLANCAS,PIEDRAS BLANCAS,NaN
280,,,,"[283, 134]",[134],"[281, 121]","[89, 14, 48]",,SIRENA LA ISLA,LA MADERA Y MONTANITA,NaN


# --

# --

### Diccionario variable - nivel

- **Fecha y Hora**
- **N actual** Se consulta a la base de datos en 192.168.1.74 usando el modulo CPR.
- **Nombre riesgo actual** - Se consulta de un binario que se debe actualizar cuando hayan nuevas estaciones.
                        '../series/op/risklevels_all_op.msg'
- **Color riesgo actual**
- **Nmáx 30 min**  - Se consulta del modelo de Esneider, se cogen las estaciones quehayan.
- **Nombre riesgo 30min**
- **Color**


**Notas**:

- Siempre se lee el último nivel del .csv de mario, si la estación no tiene datos el programa se queda leyendo  el último registrado.

¿Mario que hace con los faltantes?¿nan?

- Si es nan, todos los campos son nan y el color es gris.

- ** la estaciones en est_n_alarmas deben estar incluídas en est_n**

- ** las de est_pronos_al deben estar en n_pronos**


Qué y cuándo se debe actualizar:

- se instale nuevas estaciones de nivel para alarmas! -> agregar a n_alarmas, actualizar '../series/op/estaciones_datos_op.msg'
- se cambien los niveles de alerta de las estaciones! -> '../series/op/risklevels_all_op.msg'
- cuando se agreguen estaciones de nivel de alarmas al modelo estadístico -> n_pronos_alarmas

#### Nivel Actual + irisk levels + N30m + risk levels 30m

In [61]:
def get_levelrisk(listconfig,dfconfig,risklevels_all):
    n_alarmas=np.unique(np.sort(np.hstack(dfconfig['EstNivel'])))###########ASIGNACION DE ESTACIONES.

    ### inputs para risk_actual
    risk_names=al.get_line(listconfig,'risk_names')#['Level Action', 'Minor Flood', 'Moderate Flood','Major Flood','Major to the major!']
    risk_colors=al.get_line(listconfig,'risk_colors')#['#008000','#FFA500','#FF4500','#4B0082','#4B0082']
    #se leen los niveles de riesgo
#     risklevels_all=pd.read_csv(al.get_ruta(listconfig,'ruta_allrisklevels'))###################################

    #for para todos las estaciones
    n_act={};risk_act={};riskcolor_act={}
    for est in n_alarmas:
        #se obtiene guarda el n_act.
        self= cpr.Nivel(est)
        start=dt.datetime.now().strftime('%Y-%m-%d %H:%M')
        end=dt.datetime.now().strftime('%Y-%m-%d %H:%M')
        level=self.get_level(start,end,offset='dinamico')['nivel']
        n_act.update({str(est):float(level.values)})    
        #si es nan
        if np.isnan(level[level.index[0]]):
            risk_act.update({str(est):'nan'})
            riskcolor_act.update({str(est):'#A9A9A9'})
        #si no, se obtiene el risk actual.
        else:
            #se leen los risklevels de la estacion y se hace lista
            rl_list=list(np.array(risklevels_all.loc[risklevels_all.index==est])[0])
            #se le agrega el n_actual   
            rl_list.append(n_act[str(est)])
            #se organiza de menor a mayor
            rl_ordered=np.argsort(rl_list)
            #en que posicion quedo el ultimo valor agregado a la lista -> risk
            risk=int(np.where(rl_ordered==4)[0])
            #se guarda en el dic
            risk_act.update({str(est):risk_names[risk]})
            riskcolor_act.update({str(est):risk_colors[risk]})

    #### Nivel 30m
    ### inputs para N30m
    #--------------------------
    # se lee la info del pronostico 30m
    # ruta_npronos='/media/nicolas/Home/Jupyter/Esneider/modelo_crecidas/pronostico_niveles.bin'
    f=open(al.get_ruta(listconfig,'ruta_estadistico'))
    n_pronos=pickle.load(f)
    f.close()
    n_pronos=pd.DataFrame(n_pronos)
    #se lee la info de dfconfig
    n_pronos_alarmas=np.hstack(dfconfig['EstN30m'])
    #se leen las 
    n_pronos_alarmas=map(int,n_pronos_alarmas[np.where(n_pronos_alarmas)[0]])
    est_pronos_ordered=np.sort(n_pronos[0])
    #pos de alarmas en n_pronos
    pos_pronos_al=est_pronos_ordered.searchsorted(n_pronos_alarmas)

    #todas las estaciones.
    n_pronostico={};risk_pronos={};riskcolor_pronos={}
    for pos_pronos_est in pos_pronos_al:
        #filas en n_pronos que concuerdan con alarmas}
        pro=n_pronos.loc[n_pronos[0]==est_pronos_ordered[pos_pronos_est]]
        #se obtiene y guarda el n_pronos.
        pronos_list=[float(x)for x in np.array(pro)[0][1:]]
        pronos=pronos_list[2]
        n_pronostico.update({str(est_pronos_ordered[pos_pronos_est]):pronos})

        #si es nan - por algun motivo razon o circunstacia.
        if np.isnan(pronos):
            risk_pronos.update({str(est_pronos_ordered[pos_pronos_est]):'nan'})
            riskcolor_pronos.update({str(est_pronos_ordered[pos_pronos_est]):'#A9A9A9'})
        else:
            #se obtiene el risk_actual
#             rl_=risklevels_all.loc[risklevels_all['codigo']==est_pronos_ordered[pos_pronos_est]]

            #lista obviando la col='codigo'
#             rl_list=[float(x)for x in np.array(rl)[0][2:]]
            rl_list=list(np.array(risklevels_all.loc[risklevels_all.index==est_pronos_ordered[pos_pronos_est]])[0])
            #se le agrega el n_actual   
            rl_list.append(pronos)
            #se organiza de menor a mayor
            rl_ordered=np.argsort(rl_list)
            #en que posicion quedo -> risk
            risk=int(np.where(rl_ordered==4)[0])
            #se guarda en el dic
            risk_pronos.update({str(est_pronos_ordered[pos_pronos_est]):risk_names[risk]})
            riskcolor_pronos.update({str(est_pronos_ordered[pos_pronos_est]):risk_colors[risk]})

    #Escribe diccionario para json
    dictt={}
    #Datetime en formato '%Y-%m-%d-%H:%M'
    dictt.update({'Fecha y Hora':str(level.index[0])})
    dictt.update({'Nivel Actual':n_act})
    dictt.update({'Riesgo Actual':risk_act})
    dictt.update({'Color Riesgo Actual':riskcolor_act})
    dictt.update({'Nivel Max. prox. 30min':n_pronostico})
    dictt.update({'Riesgo Max. prox. 30min':risk_pronos})
    dictt.update({'Color Riesgo Max. prox. 30min':riskcolor_pronos})

    #Escribe json
    df=pd.DataFrame.from_dict(dictt)
    df.to_json(al.get_ruta(listconfig,'ruta_JSONnivel'))

In [62]:
get_levelrisk(listconfig,dfconfig,risklevels_all)

In [12]:
#### Lee Json
pd.read_json(al.get_ruta(listconfig,'ruta_JSONnivel'))
#!!!! La info del N30m se lee en la segunda de las uqe tienen repetidas.

,Color Riesgo Actual,Color Riesgo Max. prax. 30min,Fecha y Hora,Nivel Actual,Nivel Max. prax. 30min,Riesgo Actual,Riesgo Max. prax. 30min
106,#008000,None,2018-02-11 21:00:00,29.23,NaN,LevelAction,None
106,None,#008000,2018-02-11 21:00:00,NaN,0.0,None,LevelAction
108,#008000,None,2018-02-11 21:00:00,68.97,NaN,LevelAction,None
108,None,#008000,2018-02-11 21:00:00,NaN,0.0,None,LevelAction
124,#008000,None,2018-02-11 21:00:00,26.28,NaN,LevelAction,None
173,#A9A9A9,None,2018-02-11 21:00:00,NaN,NaN,nan,None
179,#A9A9A9,None,2018-02-11 21:00:00,NaN,NaN,nan,None
179,None,#008000,2018-02-11 21:00:00,NaN,0.0,None,LevelAction
186,#008000,None,2018-02-11 21:00:00,24.42,NaN,LevelAction,None
246,#A9A9A9,None,2018-02-11 21:00:00,NaN,NaN,nan,None


In [129]:
dictt

{'Color Riesgo Actual': {'106': '#008000',
  '108': '#008000',
  '124': '#008000',
  '173': '#A9A9A9',
  '179': '#A9A9A9',
  '186': '#008000',
  '246': '#A9A9A9',
  '251': '#008000',
  '259': '#008000',
  '265': '#008000',
  '90': '#008000'},
 'Color Riesgo Max. prax. 30min': {'106.0': '#008000',
  '108.0': '#008000',
  '179.0': '#008000'},
 'Fecha y Hora': '2018-02-11 20:55:00',
 'Nivel Actual': {'106': 29.970000000000027,
  '108': 69.29000000000002,
  '124': 25.379999999999995,
  '173': nan,
  '179': nan,
  '186': 24.55000000000001,
  '246': nan,
  '251': 15.530000000000001,
  '259': 3.4399999999999977,
  '265': 22.00999999999999,
  '90': 71.92000000000002},
 'Nivel Max. prax. 30min': {'106.0': 0.0, '108.0': 0.0, '179.0': 0.0},
 'Riesgo Actual': {'106': 'LevelAction',
  '108': 'LevelAction',
  '124': 'LevelAction',
  '173': 'nan',
  '179': 'nan',
  '186': 'LevelAction',
  '246': 'nan',
  '251': 'LevelAction',
  '259': 'LevelAction',
  '265': 'LevelAction',
  '90': 'LevelAction'},
 'R

# ---

### Pluvio

In [167]:
def get_pluviorisk(listconfig,dfconfig):
    # estaciones pluvio
    est_p=np.sort(np.hstack(dfconfig['EstMeteoAdentro']))
    est_p=map(int,est_p[np.where(est_p)[0]])
    est_p1=np.sort(np.hstack(dfconfig['EstMeteoAfuera']))
    est_p.append(map(int,est_p1[np.where(est_p1)[0]]))
    est_p2=np.sort(np.hstack(dfconfig['EstPluvioAdentro']))
    est_p.append(map(int,est_p2[np.where(est_p2)[0]]))
    est_p3=np.sort(np.hstack(dfconfig['EstPluvioAfuera']))
    est_p.append(map(int,est_p3[np.where(est_p3)[0]]))
    # forecast
    res_pluvioforecast=al.get_ruta(listconfig,'ruta_pluvioforecast')
    #Se lee forecast estadistico escenario medio.
    f = open(res_pluvioforecast+'_cast_normal.rain','r')
    cast_normal = pickle.load(f)
    f.close()
    # est pluvio_forecast
    est_pluvio=map(str,np.unique(np.hstack(est_p)))
    p_out=[i in cast_normal.columns for i in est_pluvio]
    p_in=np.where(p_out)[0]
    est_pluvio=np.take(est_pluvio,p_in)
    #df pluvioforecast con estaciones de alarmas
    df_p60m=cast_normal[est_pluvio]
    ##Iteracion
    p_act={};p60m={};p60m_time={}
    for est in np.unique(np.hstack(est_p)):
        #se obtiene y guardar pluvio actual
        self= cpr.Pluvio(est)
        start=dt.datetime.now().strftime('%Y-%m-%d %H:%M')
        end=dt.datetime.now().strftime('%Y-%m-%d %H:%M')
        pluvio=self.read_pluvio(start,end)
        p_act.update({str(est):float(pluvio.values)})
        #se obtiene y guarda el maximo de pluvio en la proxima hora.
        if str(est) in est_pluvio:
            p60m.update({str(est):df_p60m[str(est)].max()})
            p60m_time.update({str(est):str(df_p60m[str(est)].argmax())})
            
    #Escribe diccionario para json
    dictt={}
    #Datetime en formato '%Y-%m-%d-%H:%M'
    dictt.update({'Fecha y Hora':str(pluvio.index[0])})
    dictt.update({'Pluvio Actual':p_act})
    dictt.update({'Pluvio max. prox. 60  min':p60m})
    dictt.update({'Hora máx. prox. 60  min':p60m_time})
    
    #Escribe json
    df=pd.DataFrame.from_dict(dictt)
    df.to_json(al.get_ruta(listconfig,'ruta_JSONpluvio'))

In [168]:
get_pluviorisk(listconfig,dfconfig)

In [170]:
#### Lee Json
pd.read_json(al.get_ruta(listconfig,'ruta_JSONpluvio'))
#!!!! Las estaciones que no tienen info del P60m son las que tienen NaN y en hora None.

,Fecha y Hora,Hora máx. prox. 60 min,Pluvio Actual,Pluvio max. prox. 60 min
105,2018-02-11 23:10:00,None,NaN,NaN
18,2018-02-11 23:10:00,2018-02-11 23:10:00,0.0,0.0
197,2018-02-11 23:10:00,None,NaN,NaN
249,2018-02-11 23:10:00,None,NaN,NaN
253,2018-02-11 23:10:00,2018-02-11 23:10:00,0.0,0.0
261,2018-02-11 23:10:00,2018-02-11 23:10:00,NaN,0.0
267,2018-02-11 23:10:00,2018-02-11 23:10:00,0.0,0.0
29,2018-02-11 23:10:00,2018-02-11 23:10:00,0.0,0.0
3,2018-02-11 23:10:00,2018-02-11 23:10:00,NaN,0.0
33,2018-02-11 23:10:00,2018-02-11 23:10:00,NaN,0.0


### GeoJSONs Pluvio-Nivel
Funcion que escriben GeoJsons de puntos que serán estáticos.

In [4]:
def writeGeoJSON_points(rutaconfig,dfconfig_key,verbose=True):
    #INPUTS
    #lee configfile
    listconfig=al.get_rutesList(rutaconfig)
    #dfconfig
    dfconfig=pd.read_json(al.get_ruta(listconfig,'ruta_JSONinfosirenas'))
    #Estaciones.
    if dfconfig_key== 'EstNivel': #para que consulte las cosas de NIVEL
        #rutas
        rutaJSON=al.get_ruta(listconfig,'ruta_JSONnivel')
        rutaGeoJSON=al.get_ruta(listconfig,'ruta_GeoJSONnivel')
        #consulta a BD SIATA de la info de tabla estaciones para red='nivel'.
        infoestaciones = cpr.Nivel().infost
        #estaciones
        estaciones=np.sort(np.hstack(dfconfig[dfconfig_key]))
        estaciones=map(int,estaciones[np.where(estaciones)[0]])
    else: #para que consulte las cosas de PLUVIO
        #rutas
        rutaJSON=al.get_ruta(listconfig,'ruta_JSONpluvio')
        rutaGeoJSON=al.get_ruta(listconfig,'ruta_GeoJSONpluvio')
        #consulta a BD SIATA de la info de tabla estaciones para red='pluvio'.
        infoestaciones = cpr.Pluvio().infost
        # se definen estaciones
    #     est_p=np.sort(np.hstack(dfconfig['EstMeteoAdentro']))
    #     est_p=map(int,est_p[np.where(est_p)[0]])
    #     est_p1=np.sort(np.hstack(dfconfig['EstMeteoAfuera']))
    #     est_p.append(map(int,est_p1[np.where(est_p1)[0]]))
        est_p2=np.sort(np.hstack(dfconfig['EstPluvioAdentro']))
        est_p=map(int,est_p2[np.where(est_p2)[0]])
        est_p3=np.sort(np.hstack(dfconfig['EstPluvioAfuera']))
        est_p.append(map(int,est_p3[np.where(est_p3)[0]]))
        estaciones=map(str,np.unique(np.hstack(est_p)))        

    #GEOJSON.
    #propiedades estacion
    features=[]
    for est in estaciones:
        properties={
                "Codigo":str(est),
                "Nombre":infoestaciones['NombreEstacion'][infoestaciones.index==est][est],
                "Ciudad":infoestaciones['Ciudad'][infoestaciones.index==est][est],
                "jsondata":rutaJSON           
                }
        geometry={"type":"Point","coordinates":[float(infoestaciones['Longitude'][infoestaciones.index==est][est]),float(infoestaciones['Latitude'][infoestaciones.index==est][est])]
                }
        feat_est={}
        feat_est.update({"type":"Feature"})
        feat_est.update({"geometry":geometry})
        feat_est.update({"properties":properties})
        #se van guardando para todas las estaciones.
        features.append(feat_est)
    dicGeoJSON={}
    dicGeoJSON.update({"features":features})
    dicGeoJSON.update({"type":"FeatureCollection"})

    #Escribe GeoJSON incial
    f = open(rutaGeoJSON,'w')
    f.write(str(dicGeoJSON))
    f.close()
    #lo lee de nuevo para poder aplicar replace.
    f = open(rutaGeoJSON,'r')
    filedata = f.read()
    f.close()
    #replace de comilla sencilla a doble para que pueda servir el GeoJSON
    newdata = filedata.replace("'",'"')
    #escribe nuevamente
    f = open(rutaGeoJSON,'w')
    f.write(newdata)
    f.close()
    
    if verbose:
        print 'Aviso: Se genera archivo de puntos GeoJSON en '+rutaGeoJSON

In [60]:
writeGeoJSON_points(rutaconfig,'EstNivel')

Aviso: Se genera archivo de puntos GeoJSON en /media/nicolas/Home/Jupyter/Soraya/git/Alarmas/04_web_hidrologia/hidrologia/results/01_estaticos/Geo_nivel.geojson


### KMLs Pluvio-Nivel
Funcion que escriben KML de puntos que serán estáticos.

### Write KML

In [236]:
Data={}
for est in estaciones:
    Data_est={}
#     Data_est.update({FieldsNames[0]:infoestaciones['NombreEstacion'][infoestaciones.index==est][est]})
    Data_est.update({FieldsNames[0]:str(est)})
    Data_est.update({FieldsNames[1]:infoestaciones['Ciudad'][infoestaciones.index==est][est]})
    Data_est.update({FieldsNames[2]:str(np.array(jsonnivel[jsonnivel.index==est]['Nivel Actual'])[0])})
    Data.update({str(est):Data_est})
Data.update({FieldsNames[3]:str(jsonnivel[jsonnivel.index==estaciones[0]]['Fecha y Hora'][estaciones[0]])})

In [264]:
def writeKML(listconfig,dfconfig_key,verbose=True):
    ''' Escribe '''
    #dfconfig
    dfconfig=pd.read_json(al.get_ruta(listconfig,'ruta_JSONinfosirenas'))
    #Estaciones.
    if dfconfig_key== 'EstNivel': #para que consulte las cosas de NIVEL
        #rutas
        rutaJSON=al.get_ruta(listconfig,'ruta_JSONnivel')
        rutaKML=al.get_ruta(listconfig,'ruta_KMLnivel')
        #consulta a BD SIATA de la info de tabla estaciones para red='nivel'.
        infoestaciones = cpr.Nivel().infost
        #estaciones
        estaciones=np.sort(np.hstack(dfconfig[dfconfig_key]))
        estaciones=map(int,estaciones[np.where(estaciones)[0]])
        #Alista datos para KML - NIVEL
        JSON=pd.read_json(rutaJSON)
        #Campos a agregar a KML - NIVEL.
        FieldsNames=['Codigo','Ciudad','Nact','fecha_act']
        FieldsTypes=['string','string','string','string']
        SchemaName='RedNivel'
        #datos
        Data={}
        for est in estaciones:
            Data_est={}
            Data_est.update({FieldsNames[0]:str(est)})
            Data_est.update({FieldsNames[1]:infoestaciones['Ciudad'][infoestaciones.index==est][est]})
            Data_est.update({FieldsNames[2]:str(np.array(JSON[JSON.index==est]['Nivel Actual'])[0])})
            Data.update({str(est):Data_est})
        Data.update({FieldsNames[3]:str(JSON[JSON.index==estaciones[0]]['Fecha y Hora'][estaciones[0]])})
    else: #para que consulte las cosas de PLUVIO
        #rutas
        rutaJSON=al.get_ruta(listconfig,'ruta_JSONpluvio')
        rutaKML=al.get_ruta(listconfig,'ruta_KMLpluvio')
        #consulta a BD SIATA de la info de tabla estaciones para red='pluvio'.
        infoestaciones = cpr.Pluvio().infost
        # se definen estaciones
    #     est_p=np.sort(np.hstack(dfconfig['EstMeteoAdentro']))
    #     est_p=map(int,est_p[np.where(est_p)[0]])
    #     est_p1=np.sort(np.hstack(dfconfig['EstMeteoAfuera']))
    #     est_p.append(map(int,est_p1[np.where(est_p1)[0]]))
        est_p2=np.sort(np.hstack(dfconfig['EstPluvioAdentro']))
        est_p=map(int,est_p2[np.where(est_p2)[0]])
        est_p3=np.sort(np.hstack(dfconfig['EstPluvioAfuera']))
        est_p.append(map(int,est_p3[np.where(est_p3)[0]]))
        estaciones=map(str,np.unique(np.hstack(est_p))) 
    

    #KML
    #BEGINNING
    KML=['<?xml version="1.0" encoding="utf-8" ?>\n',
     '<kml xmlns="http://www.opengis.net/kml/2.2">\n',
     '<Document id="root_doc">\n']
    #SCHEMA
    KML.append('<Schema name="'+SchemaName+'" id="'+SchemaName+'">\n')
    for index,Fname in enumerate(FieldsNames):
        KML.append('  <SimpleField name="'+Fname+'" type="'+FieldsTypes[index]+'"></SimpleField>\n')
    KML.append('</Schema>\n')
    #FOLDER
    KML.append('<Folder><name>'+SchemaName+'</name>\n')
    #PLACEMARKS
    PlmkNames=[infoestaciones['NombreEstacion'][infoestaciones.index==est][est] for est in estaciones]
    for index,est in enumerate(estaciones):
        KML.append('  <Placemark>\n')

        KML.append('<name>"'+PlmkNames[index]+'"</name>\n')
        KML.append('    <ExtendedData><SchemaData schemaUrl="#RedAirePM25">\n')
        #itera sobre campos agregados
        for Fname in (FieldsNames[:-1]):
            KML.append('        <SimpleData name="'+Fname+'">'+Data[str(est)][Fname]+'</SimpleData>\n')
        KML.append('    </SchemaData></ExtendedData>\n')
        KML.append('      <Point><coordinates>'+infoestaciones['Longitude'][infoestaciones.index==est][est]+','+infoestaciones['Latitude'][infoestaciones.index==est][est]+'</coordinates></Point>\n')
        KML.append('  </Placemark>\n')  
    #CLOSING
    KML.append('</Folder>\n')
    KML.append('</Document></kml>')
    
    #WRITE FILE!
    f = open(rutaKML,'w')
    f.writelines(KML)
    f.close()
    
    if verbose:
        print 'Aviso: Se genera archivo de puntos KML en '+rutaKML

In [265]:
 writeKML(listconfig,dfconfig_key)

Aviso: Se genera archivo de puntos KML en /media/nicolas/Home/Jupyter/Soraya/git/Alarmas/04_web_hidrologia/hidrologia/results/02_operacionales/nivel.kml


# ---

### Radar

MONTAR TODAS LOS .nc DE LAS CUENCAS Y CRONEAR LA LLUVIA!... SACARLAS DE DONDE LOS GENERA MARIO A PARTIR DEL UMBRAL DE NIVEL.

REVISAR OTRAS GRÁFICAS QUE SE HABÍAN CONVERSADO CON ESNEIDER PARA MONTAR ACÁ.


# Datos base

### Niveles de riesgo

In [31]:
# codigo de la estacion.
codeest=est_n_ordered[0]
# coneccion a bd con usuario operacional
host   = '192.168.1.74'
user   = 'siata_Oper'
passwd = 'si@t@64512_operacional'
bd     = 'siata'
#Consulta a tabla estaciones
Estaciones="SELECT codigo,action_level,minor_flooding,moderate_flooding,major_flooding  FROM estaciones WHERE codigo=("+str(codeest)+")"
dbconn = MySQLdb.connect(host, user,passwd,bd)
db_cursor = dbconn.cursor()
db_cursor.execute(Estaciones)
result = np.array(db_cursor.fetchall())
#holding
risklevels_all=pd.DataFrame(result,columns=['codigo','action_level','minor_f','moderate_f','major_f'])

In [32]:
risklevels_all

,codigo,action_level,minor_f,moderate_f,major_f
0,90,93.0,171.0,202.0,259.0


In [33]:
for ind,est in enumerate(est_n_ordered):
    try:
        # codigo de la estacion.
        codeest=est_n_ordered[ind+1]
        # coneccion a bd con usuario operacional
        host   = '192.168.1.74'
        user   = 'siata_Oper'
        passwd = 'si@t@64512_operacional'
        bd     = 'siata'
        #Consulta a tabla estaciones
        Estaciones="SELECT codigo,action_level,minor_flooding,moderate_flooding,major_flooding  FROM estaciones WHERE codigo=("+str(codeest)+")"
        dbconn = MySQLdb.connect(host, user,passwd,bd)
        db_cursor = dbconn.cursor()
        db_cursor.execute(Estaciones)
        result = np.array(db_cursor.fetchall())
        #holding
        risklevels=pd.DataFrame(result,columns=['codigo','action_level','minor_f','moderate_f','major_f'])
        risklevels_all=risklevels_all.append(risklevels)
    except:
        pass

In [37]:
risklevels_all

,codigo,action_level,minor_f,moderate_f,major_f
0,90,93.0,171.0,202.0,259.0
0,91,93.0,126.0,353.0,501.0
0,92,87.0,123.0,180.0,227.0
0,93,92.0,132.0,202.0,285.0
0,94,92.0,172.0,228.0,288.0
0,95,-999.0,-999.0,-999.0,-999.0
0,96,82.0,106.0,162.0,206.0
0,97,-999.0,-999.0,-999.0,-999.0
0,98,233.0,272.0,305.0,375.0
0,99,140.0,210.0,260.0,350.0


In [41]:
#Se guarda info
risklevels_all.to_csv('op/datos_base/risklevels_all_op.csv')

### Históricos offset

In [43]:
# codigo de la estacion.
codeest=est_n_ordered[0]
# coneccion a bd con usuario operacional
host   = '192.168.1.74'
user   = 'siata_Oper'
passwd = 'si@t@64512_operacional'
bd     = 'siata'

#Consulta a tabla historico_bancallena_offset
#select fecha_hora,offset from historico_bancallena_offset where codigo='106'
offsets="SELECT codigo,fecha_hora,offset FROM historico_bancallena_offset WHERE codigo=("+str(codeest)+")"
dbconn = MySQLdb.connect(host, user,passwd,bd)
db_cursor = dbconn.cursor()
db_cursor.execute(offsets)
result1 = np.array(db_cursor.fetchall())
#holding
offset_all=pd.DataFrame(result1,columns=['codigo','date','offset'])

In [44]:
offset_all

,codigo,date,offset
0,90,2013-12-03 12:00:00,393
1,90,2017-05-16 14:00:00,409
2,90,2017-05-16 14:40:00,408
3,90,2017-05-16 15:00:00,409
4,90,2017-10-05 12:25:00,321.94
5,90,2017-12-19 16:25:00,343


In [47]:

for ind,est in enumerate(est_n_ordered):
    try:
        # codigo de la estacion.
        codeest=est_n_ordered[ind+1]
        # coneccion a bd con usuario operacional
        host   = '192.168.1.74'
        user   = 'siata_Oper'
        passwd = 'si@t@64512_operacional'
        bd     = 'siata'

        #Consulta a tabla historico_bancallena_offset
        #select fecha_hora,offset from historico_bancallena_offset where codigo='106'
        offsets="SELECT codigo,fecha_hora,offset FROM historico_bancallena_offset WHERE codigo=("+str(codeest)+")"
        dbconn = MySQLdb.connect(host, user,passwd,bd)
        db_cursor = dbconn.cursor()
        db_cursor.execute(offsets)
        result1 = np.array(db_cursor.fetchall())
        #holding
        offsetH=pd.DataFrame(result1,columns=['codigo','date','offset'])
        offset_all=offset_all.append(offsetH)
    except:
        pass

In [125]:
offset_all

,codigo,date,offset
0,90,2013-12-03 12:00:00,393
1,90,2017-05-16 14:00:00,409
2,90,2017-05-16 14:40:00,408
3,90,2017-05-16 15:00:00,409
4,90,2017-10-05 12:25:00,321.94
0,91,2014-03-14 12:00:00,715
0,92,2012-06-14 12:00:00,525
1,92,2017-03-31 11:20:00,525
0,93,2012-04-13 12:00:00,985
0,94,2012-04-11 12:00:00,1075


In [49]:
#Guardar info
offset_all.to_csv('op/datos_base/offsetsH_all_op.csv')


### Chequear hexadecimal de colores 

lcolors=['g','orange','orangered','indigo']
import matplotlib
print matplotlib.colors.cnames['darkgray']

estaciones_datos_all.loc[estaciones_datos_all['codigo'].isin(est_nordered_list)].shape

# --

#### Coordenadas y nombres de estaciones

In [81]:
codeest=est_n_ordered[0]
# coneccion a bd con usuario operacional
host   = '192.168.1.74'
user   = 'siata_Oper'
passwd = 'si@t@64512_operacional'
bd     = 'siata'
#Consulta a tabla estaciones
Estaciones="SELECT codigo,longitude,latitude,nombreestacion  FROM estaciones WHERE codigo=("+str(codeest)+")"
dbconn = MySQLdb.connect(host, user,passwd,bd)
db_cursor = dbconn.cursor()
db_cursor.execute(Estaciones)
result = np.array(db_cursor.fetchall())
estaciones_datos_all=pd.DataFrame(result,columns=['codigo','longitud','latitud','nombreestacion'])

In [82]:
for ind,est in enumerate(est_n_ordered):
    try:
        # codigo de la estacion.
        codeest=est_n_ordered[ind+1]
        # coneccion a bd con usuario operacional
        host   = '192.168.1.74'
        user   = 'siata_Oper'
        passwd = 'si@t@64512_operacional'
        bd     = 'siata'
        #Consulta a tabla estaciones
        Estaciones="SELECT codigo,longitude,latitude,nombreestacion  FROM estaciones WHERE codigo=("+str(codeest)+")"
        dbconn = MySQLdb.connect(host, user,passwd,bd)
        db_cursor = dbconn.cursor()
        db_cursor.execute(Estaciones)
        result = np.array(db_cursor.fetchall())
        #holding
        estaciones_datos=pd.DataFrame(result,columns=['codigo','longitud','latitud','nombreestacion'])
        estaciones_datos_all=estaciones_datos_all.append(estaciones_datos)
    except:
        pass

In [ ]:
estaciones_datos_all=estaciones_datos_all.drop_duplicates(subset='codigo')

estaciones_datos_all.to_msgpack('../series/estaciones_datos_op.msg')

## no se si sevirá

# Nivel

### Diccionario estático


- Nombre estacion
- Coordenadas
- Shp cuenca
- AGREGAR: kml de estacions de nivel!!!!

Las coordenadas de las alarmas con sus nombres, están en este msg:
     
        '../series/op/nombres_sirenas_op.msg'
        
Este se va a actualizar sólo cuando se agreguen nuevas estaciones de nivel asociadas a alarmas y sirenas, la mayoría del tiempo es estático.

In [87]:
#------------------------------------
##coordenadas y nombres de estaciones
#------------------------------------

est_datos=pd.read_msgpack('../series/op/estaciones_datos_op.msg')#####################################################

# se crean dicts.
lon_lat={};nombreestacion={};nombrealarma={};nombrecorriente={}
for est in n_alarmas:
    #se obtiene y guarda los datos de las n_alarmas
    datos=np.array(est_datos.loc[est_datos['codigo']==str(est)])[0][1:]
    lon_lat.update({str(est):(datos[0],datos[1])})
    nombreestacion.update({str(est):datos[2]})

In [89]:
#Escribe diccionario para json
dictt={}
dictt.update({'lon_lat':lon_lat})
dictt.update({'Nombre Estacion':nombreestacion})

df=pd.DataFrame.from_dict(dictt)
df.to_json('prueba_jsonnivel_estatico_op.json')

In [91]:
dictt

{'Nombre Estacion': {'108': 'Santa Rita - San Antonio de Prado',
  '135': 'Q. La loca - Nivel',
  '173': 'Q. Picacha - Aguas Frias',
  '179': 'La Inmaculada - Nivel',
  '186': 'Q. La Gallinaza',
  '246': 'Q. La Raya - Nivel',
  '251': 'Q.  la L\xf3pez - Nivel',
  '259': 'Andaluc\xeda Caldas - Nivel'},
 'lon_lat': {'108': ('-75.651535', '6.190568'),
  '135': ('-75.5608889', '6.3236944'),
  '173': ('-75.639668', '6.231477'),
  '179': ('-75.633503', '6.136677'),
  '186': ('-75.3277100', '6.4329150'),
  '246': ('-75.64090', '6.11194'),
  '251': ('-75.32507', '6.42794'),
  '259': ('-75.62651', '6.09259')}}

In [ ]:
dflol=pd.read_json('../../../series/op/prueba_jsonnivel_estatico_op.json')
dflol

# !!!Not longer useful

#see leen las rutas
rutes=glob.glob(al.get_ruta(listconfig,'ruta_nivel')+'*')
#se deduce la estacion de cada archivo para luego organizarlos en orden
est_n=np.array([int(rute.split('/')[-1].split('_')[0]) for rute in rutes])
#orden para organizarlos
order_est=est_n.argsort()
#se ordenan 
est_n_ordered=np.take(est_n,order_est)
rutes_ordered=np.take(rutes,order_est)
#est - alarmas
n_alarmas=np.sort(np.hstack(dfconfig['EstNivel']))###########ASIGNACION DE ESTACIONES.
#pos de los archivos correspondientes a alarmas - se dan todas esas vueltas para evitar un for para todas las est de nivel
pos_al=est_n_ordered.searchsorted(n_alarmas) #****REVISAAR, esta función busca las pos en est_n_ordered donde deberían estar n_alarmas para estar organizados.

#### Trying to write kml from python.

In [83]:
kml=simplekml.Kml()
names=[]
for est in estaciones:
    name=kml.newpoint(name=infoestaciones['NombreEstacion'][infoestaciones.index==est][est],
                coords=[(float(infoestaciones['Longitude'][infoestaciones.index==est][est]),float(infoestaciones['Latitude'][infoestaciones.index==est][est]))])
    names.append(name)

In [84]:
print kml.kml()

<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://www.opengis.net/kml/2.2" xmlns:gx="http://www.google.com/kml/ext/2.2">
    <Document id="feat_15">
        <Placemark id="feat_16">
            <name>Colegio Campestre el Encanto</name>
            <Point id="geom_12">
                <coordinates>-75.661333,6.1953667,0.0</coordinates>
            </Point>
        </Placemark>
        <Placemark id="feat_17">
            <name>3 Aguas - Nivel</name>
            <Point id="geom_13">
                <coordinates>-75.63526,6.09602,0.0</coordinates>
            </Point>
        </Placemark>
        <Placemark id="feat_18">
            <name>Santa Rita - San Antonio de Prado</name>
            <Point id="geom_14">
                <coordinates>-75.651535,6.190568,0.0</coordinates>
            </Point>
        </Placemark>
        <Placemark id="feat_19">
            <name>Caldas-Unidad Deportiva</name>
            <Point id="geom_15">
                <coordinates>-75.633048,6.088251,0

kml.newschemaope

print simplekml.SimpleField(name='Nact',type='float')

simplekml.SimpleData(name='Nact',value='0.27')

In [85]:
kml1=simplekml.SimpleField(name='Nact',type='float')

# ---

In [61]:
from pykml.factory import KML_ElementMaker as KML
from pykml.factory import ATOM_ElementMaker as ATOM
from pykml.factory import GX_ElementMaker as GX
from lxml import etree
from pykml import parser
from pykml.factory import nsmap

In [57]:
rutakml='/media/nicolas/Home/Jupyter/Soraya/git/Alarmas/04_web_hidrologia/hidrologia/op/kml/ubicacion_gral/estnivel_al.kml'

In [58]:
with open(rutakml) as f:
    folder = parser.parse(f).getroot().Document.Folder

# for pm in folder.Placemark:
#     print(pm.name)

AttributeError: no such child: {http://www.opengis.net/kml/2.2}name

In [66]:
namespace = {"ns": nsmap[None]}

with open(rutakml) as f:
    root = parser.parse(f).getroot()
    pms = root.findall(".//ns:Placemark", namespaces=namespace)

In [80]:
 from os import path
kml_file = path.join(rutakml)
    
with open(kml_file) as f:
    doc = parser.parse(f)

In [82]:
kml_file

'/media/nicolas/Home/Jupyter/Soraya/git/Alarmas/04_web_hidrologia/hidrologia/op/kml/ubicacion_gral/estnivel_al.kml'

In [ ]:
name_object = KML.name("Hello World!")

pm1 = KML.Placemark(
            KML.name("Hello World!"),
            KML.Point(
               KML.coordinates("-64.5253,18.4607")
             )
           )

In [51]:
print etree.tostring(pm1, pretty_print=True)

<Placemark xmlns:gx="http://www.google.com/kml/ext/2.2" xmlns:atom="http://www.w3.org/2005/Atom" xmlns="http://www.opengis.net/kml/2.2">
  <name>Hello World!</name>
  <Point>
    <coordinates>-64.5253,18.4607</coordinates>
  </Point>
</Placemark>

